In [2]:
import torch
import torchvision
import torch.nn as nn
from torchvision import datasets
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#data transformation
transform_train = transforms.Compose([
  transforms.RandomCrop(32, padding=4),
  transforms.RandomHorizontalFlip(),
  transforms.ToTensor(),
  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

#importing cifar10 dataset
train_dataset = datasets.CIFAR100(root='data', train=True, transform=transform_train, download=True)
test_dataset = datasets.CIFAR100(root='data', train=False, transform=transform_test)

#dataloaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=4)

100%|██████████| 169001437/169001437 [00:02<00:00, 66740590.58it/s]


Extracting data/cifar-100-python.tar.gz to data


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [4]:
#defining dense block in DenseNet
'''
Each layer consists of batch normalization, ReLU activation, depthwise convolution and pointwise convolution.
Forward pass concatenates the outputs of each layer and next one recieves features from all the preceding layers.
'''
class DenseBlock(nn.Module):
  def __init__(self, in_channels, growth_rate):
    super().__init__()

    layers = []
    for _ in range(4):  # Adjust number of layers as needed
        layers.extend([
          nn.BatchNorm2d(in_channels),
          nn.ReLU(inplace=True),
          nn.Conv2d(in_channels, growth_rate, kernel_size=3, padding=1,bias=False),
        ])
        in_channels += growth_rate

    self.layers = nn.Sequential(*layers)

  def forward(self,x):
    features = [x]

    for layer in self.layers:
      x=layer(x)
      features.append(x)
      x=torch.cat(features,dim=1)

    return x

In [5]:
#Transition layer (downsamples features -- reducing number of channels and spatial dimension while maintaining info flow)
class TransitionLayer(nn.Module):
  def __init__(self, in_channels, out_channels):
    super().__init__()

    self.convl = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
    self.pool = nn.AvgPool2d(kernel_size=2, stride=2)

  def forward(self,x):
    x=self.convl(x)
    x=self.pool(x)
    return x


In [6]:
class DenseNet(nn.Module):
  def __init__(self, growth_rate=8, num_blocks=[6,12,24,32], depthwise_conv=True):
    super().__init__()

    self.convl2=nn.Conv2d(3, growth_rate*2, kernel_size=3)
    self.dense_blocks=nn.ModuleList()

    in_channels=growth_rate*2
    for num_layers in num_blocks:
      self.dense_blocks.append(DenseBlock(in_channels, growth_rate))
      in_channels += num_layers * growth_rate
      self.dense_blocks.append(TransitionLayer(in_channels, in_channels//2))
      in_channels //= 2

    self.pool = nn.AvgPool2d(kernel_size=8, stride=8)
    self.fc = nn.Linear(in_channels, 100)

  def forward(self, x):
    x = self.convl2(x)

    for block in self.dense_blocks:
       x = block(x)

    x = self.pool(x)
    x = torch.flatten(x, 1)
    x = self.fc(x)
    return x

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = DenseNet(depthwise_conv=True).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [ ]:
#training the model
epochs = 30
train_losses, val_losses = [], []

for epoch in range(epochs):
  model.train()
  running_loss = 0.0

  for inputs, labels in train_loader:
    inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  train_loss = running_loss / len(train_loader)
  train_losses.append(train_loss)

  #validation
  model.eval()
  val_loss = 0.0

  with torch.no_grad():
    for inputs, labels in test_loader:
      inputs, labels = inputs.to(device), labels.to(device)
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      val_loss += loss.item()

  val_loss /= len(test_loader)
  val_losses.append(val_loss)


In [11]:
#evaluation
def evaluate(model, test_loader, device):
  model.eval()
  all_pred = []
  all_labels = []

  with torch.no_grad():
    for inputs, labels in test_loader:
      inputs, labels = inputs.to(device), labels.to(device)
      outputs = model(inputs)

      _, predictions = torch.max(outputs, 1)
      all_pred.extend(predictions.cpu().numpy())
      all_labels.extend(labels.cpu().numpy())

  return all_pred, all_labels

In [ ]:
#calculating evaluation metrices
predictions, labels = evaluate(model, test_loader, device)

accuracy = accuracy_score(labels, predictions)
precision = precision_score(labels, predictions, average='weighted')
recall = recall_score(labels, predictions, average='weighted')
f1 = f1_score(labels, predictions, average='weighted')

In [ ]:
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

Without dense connections (will be building second model - adding *Without* as prefix)

In [8]:
class WithoutDenseBlock(nn.Module):
  def __init__(self, in_channels, growth_rate):
      super(WithoutDenseBlock, self).__init__()

      self.layers = nn.ModuleList()
      for _ in range(4):
        self.layers.append(nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, in_channels, kernel_size=3, groups=in_channels, padding=1),
            nn.Conv2d(in_channels, growth_rate, kernel_size=1)
        ))
        in_channels += growth_rate

  def forward(self, x):
      out = x

      for layer in self.layers:
        out = layer(out)
        x = x + out #simple addition

      return x


In [9]:
class WithoutDenseNet(nn.Module):
  def __init__(self, growth_rate=12, num_blocks=[6, 12, 24, 32]):
    super().__init__()

    self.conv1 = nn.Conv2d(3, growth_rate*2, kernel_size=3)
    self.dense_blocks = nn.ModuleList()

    in_channels = growth_rate*2
    for num_layers in num_blocks:
      self.dense_blocks.append(WithoutDenseNet(in_channels, growth_rate))
      in_channels += num_layers * growth_rate
      self.dense_blocks.append(TransitionLayer(in_channels, in_channels//2))
      in_channels //= 2

    self.pool = nn.AvgPool2d(kernel_size=8, stride=8)
    self.fc = nn.Linear(in_channels, 100)

  def forward(self, x):
      x = self.conv1(x)
      for block in self.dense_blocks:
        x = block(x)
      x = self.pool(x)
      x = torch.flatten(x, 1)
      x = self.fc(x)

      return x


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = WithoutDenseNet().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [ ]:
#training the model
epochs = 30
train_losses, val_losses = [], []

for epoch in range(epochs):
  model.train()
  running_loss = 0.0

  for inputs, labels in train_loader:
    inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  train_loss = running_loss / len(train_loader)
  train_losses.append(train_loss)

  #validation
  model.eval()
  val_loss = 0.0

  with torch.no_grad():
    for inputs, labels in test_loader:
      inputs, labels = inputs.to(device), labels.to(device)
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      val_loss += loss.item()

  val_loss /= len(test_loader)
  val_losses.append(val_loss)


In [12]:
#evaluation
def evaluate(model, test_loader, device):
  model.eval()
  all_pred = []
  all_labels = []

  with torch.no_grad():
    for inputs, labels in test_loader:
      inputs, labels = inputs.to(device), labels.to(device)
      outputs = model(inputs)

      _, predictions = torch.max(outputs, 1)
      all_pred.extend(predictions.cpu().numpy())
      all_labels.extend(labels.cpu().numpy())

  return all_pred, all_labels

In [ ]:
#calculating evaluation metrices
predictions, labels = evaluate(model, test_loader, device)

accuracy = accuracy_score(labels, predictions)
precision = precision_score(labels, predictions, average='weighted')
recall = recall_score(labels, predictions, average='weighted')
f1 = f1_score(labels, predictions, average='weighted')


In [ ]:
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

Accuracy: 0.6356
Precision: 0.6481
Recall: 0.6356
F1 Score: 0.6297


In [ ]:
#CNN model (consisting of depthwise+pointwise) works better when used along with dense network.